In [31]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'../../../..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
import os
from datetime import datetime as dt
import numpy as np
import pandas as pd
import scipy.stats as scist

import matplotlib.pyplot as plt

import tensorflow as tf

from IPython.display import HTML
from IPython.core.display import Markdown as md

from tqdm.auto import tqdm, trange

import yfinance as yf
import arch
import pmdarima

from analysis.fincalcs import log_returns
from analysis.options.pricing.phat import Phat
from analysis.options.pricing.bootstrap.numpy import dbl_bs
from analysis.options.pricing.learn.utils import DataSplit
from analysis.options.pricing.learn.phatnet import PhatNet, PhatLoss, PhatMetric
from analysis.options.pricing.utils import PriceSim

## Moments ##

We will show how to find the moments, in particular the variance, of the Phat distribution. The Phat distribution is a mixture model of two Carben distributions, each weighted 50%. Thus, the pdf of the Phat distribution is found as:

$$
f(x) = \sum\limits_{i=1}^nw_if_i(x)
\\f(x) = 0.5*f_{\textit{left}}(x) + 0.5*f_{\textit{right}}(x)
$$

The Carben distribution has a piecewise density function, defined for a right-tail as:

$$
f_{\textit{right}}(x) = \left\{ \begin{array}{ll}
      \frac{1}{\gamma} f_{\mu,\sigma}(x) & \text{if } y\leq a \\
      \frac{1}{\gamma} g_{\xi,a,b}(x) & \text{if } y > a\\
\end{array} \right.
$$

And for the left

$$
f_{\textit{left}}(x) = \left\{ \begin{array}{ll}
      \frac{1}{\gamma} g_{\xi,-a,b}(-x) & \text{if } y < a \\
       \frac{1}{\gamma} f_{\mu,\sigma}(x) & \text{if } y \geq a\\
\end{array} \right.
$$

The moment generating function for a piecewise distributino is found as follows:

$$
M_x(n) = \int_{-\infty}^{+\infty}e^{nx}f_X(x)\delta x
\\ = \int_{-\infty}^{+\infty}e^{nx}\left(.5*f_{\textit{left}}(x) + .5*f_{\textit{right}}(x)\right)\delta x
$$

$$
\\ = \int_{-\infty}^{+\infty}.5*e^{nx}f_{\textit{left}}(x) + .5*e^{nx}f_{\textit{right}}(x)\delta x
\\ = .5\int_{-\infty}^{+\infty}e^{nx}f_{\textit{left}}(x)\delta x + .5\int_{-\infty}^{+\infty}e^{nx}f_{\textit{right}}(x)\delta x
\\ = .5*M_{left} + .5*M_{right}
$$

We'll start with the right term. The moment-generating function of a piecewise distribution is simply the sum of the integrals along the bounded ranges.

$$
M_{right} = \frac{1}{\gamma}\int_{-\infty}^{a}e^{nx}f_{\mu,\sigma}(x)\delta x + \frac{1}{\gamma}\int_{a}^{+\infty}e^{nx}g_{\xi,a,b}(x)\delta x
$$

The body term is a truncated normal distribution. The moments of such functions are known and available in `scipy`. We know that the generalized Pareto is always restricted the interval $(a,\infty)$, so the tail term is merely the generalized Pareto moment-generating function, MDF, scaled by $\frac{1}{\gamma}$. Thus,

$$
\frac{1}{\gamma}M_{\mu,\sigma,u}(n) + \frac{1}{\gamma}M_{\xi,a,b}(n)
$$
where: $u$ is the upper bound of the body

The second moment is the second derivative of $M$, so:

$$
\frac{d^2}{dx}\frac{1}{\gamma}M_{\mu,\sigma,u}(2) + \frac{d^2}{dx}\frac{1}{\gamma}M_{\xi,a,b}(2)
$$

The constant multiplicative survives both differntiations so that:
$$
\frac{1}{\gamma}\text{Var}_{\mu,\sigma,u} + \frac{1}{\gamma}\text{Var}_{\xi,a,b}
$$

Now for the left:

$$
M_{left}(n) = \frac{1}{\gamma}\int_{-\infty}^{a}e^{nx}g_{\xi,-a,b}(-x)\delta x
 + \frac{1}{\gamma}\int_{a}^{+\infty}e^{nx}f_{\mu,\sigma}(x)\delta x
\\=\frac{1}{\gamma}\int_{-a}^{+\infty}e^{nx}g_{\xi,-a,b}(x)\delta x
 + \frac{1}{\gamma}\int_{a}^{+\infty}e^{nx}f_{\mu,\sigma}(x)\delta x
$$

Once again, the tail portion is the same moment-generating function as generalized Pareto. And once again, the body portion is a truncated normal, this time with a lower bound, so we can find the variance as:

$$
\frac{1}{\gamma}M_{\xi,-a,b}(n) + \frac{1}{\gamma}M_{\mu,\sigma,l}(n)
$$
where: $l$ is the lower bound of the body

The second moment is the second derivative of $M$, so:

$$
\frac{d^2}{dx}\frac{1}{\gamma}M_{\xi,-a,b}(2) + \frac{d^2}{dx}\frac{1}{\gamma}M_{\mu,\sigma,l}(2)
\\=\frac{1}{\gamma}\text{Var}_{\xi,-a,b} + \frac{1}{\gamma}\text{Var}_{\mu,\sigma,l}
$$

In [46]:
phat = Phat(None, .1, .2, .2, .25)

bvar = scist.truncnorm(
    -np.inf, 
    phat.right.loc, 
    *phat.right.body.args
).var()
tvar = phat.right.tail.var()
rvar = (bvar + tvar) / phat.right.gamma

In [47]:
bvar = scist.truncnorm(
    phat.left.loc,
    np.inf, 
    *phat.left.body.args
).var()
tvar = phat.left.tail.var()
lvar = (bvar + tvar) / phat.left.gamma

In [48]:
lvar

0.4828404012432034

The variance for the Phat distribution is then: 

In [49]:
np.mean((lvar, rvar))

0.5732923559721449

This is avialable as `var` method of the Phat distribution.

In [64]:
phat.var()

0.5732923559721449